# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [116]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26

def idx_to_onehot(x, k=N+1):
  ones = torch.sparse.torch.eye(k).to(x.device)
  """ Converts the generated integers to one-hot vectors """
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])

class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=3, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [117]:
import torch.nn as nn
import torch.nn.functional as F

class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.hidden_size = hidden_size
    self.gru = nn.GRU(input_size = 27, hidden_size=hidden_size, batch_first=True)
    self.linear = nn.Linear(hidden_size, 27)

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    gru_out, _ = self.gru(x) 
    return self.linear(gru_out) 

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z.
    # You need to map those characters to one-hot encodings of shape (len(s), N + 1),
    # then feed the on-hot encodings into your network,
    # then get the output from your network,
    # then convert the output back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    # TODO
    char_to_idx = {char: idx + 1 for idx, char in enumerate('abcdefghijklmnopqrstuvwxyz')}
    char_to_idx[' '] = 0
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}
    indices = torch.tensor([char_to_idx[char] for char in s], dtype=torch.long)
     # Move inputs to the correct device (the same device as the model)
    device = next(self.parameters()).device
    indices = indices.to(device)
    
    one_hot_vector = idx_to_onehot(indices)

    one_hot_vector = one_hot_vector.unsqueeze(0)  # (1, seq_length, N+1)
    output = self.forward(one_hot_vector)
    output_idx = torch.argmax(output, dim=-1).squeeze(0) # (seq_length,)
    return ''.join([idx_to_char[idx.item()] for idx in output_idx])


## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [118]:
def test_model(model):
  model.eval()
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
      correct += int(c1 == c2)
    total += len(s) - DELAY

  return correct / total

In [119]:
import time
from tqdm.notebook import tqdm
from torch import nn
from torch import nn, optim
start_time = time.time()

HIDDEN_SIZE=64 # TODO
BATCH_SIZE=32 #TODO

num_epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train_model(model, dataloader):
  model.to(device)
  model.train()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.005)
    
  for epoch in range(num_epochs):
    epoch_loss = 0.0
        
    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False) as pbar:
      for i, (inputs, targets) in enumerate(pbar):
        inputs = idx_to_onehot(inputs).to(device)  
        targets = targets.long().to(device)  # type for CrossEntropyLoss
                
        optimizer.zero_grad()
        outputs = model(inputs)
                
        outputs = outputs.view(-1, N+1) 
        targets = targets.view(-1)  
                
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # Update running loss here
        running_loss = loss.item()
        epoch_loss += running_loss
        pbar.set_postfix({'epoch_loss': epoch_loss / (i + 1)})
        

model = GRUMemory(HIDDEN_SIZE)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)
train_model(model, dataloader)

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
print(accuracy)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Epoch 1/2:   0%|          | 0/6250 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [63]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [75]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    self.hidden_size = hidden_size
    self.max_delay = max_delay
    self.gru = nn.GRU(input_size=N+1, hidden_size=hidden_size, batch_first=True)
    self.linear = nn.Linear(hidden_size, N+1)
    self.delay_embeds = nn.Embedding(max_delay+1, hidden_size) # this would be concatenated with the output

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

    # TODO
    gru_out, _ = self.gru(x)
    delay_embeds = self.delay_embeds(delays)  # (batch_size, hidden_size)
    delay_embeds = delay_embeds.unsqueeze(1).expand(-1, x.size(1), -1)  # need (batch_size, seq_length, hidden_size)
    output = gru_out + delay_embeds
    return self.linear(output)
  

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z,
    # and an int delay - the desired output delay.
    # You need to map those characters to one-hot encodings,
    # then feed the one-hot encodings into your network,
    # then get the output from your network,
    # then convert the output back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.
    char_to_idx = {char: idx + 1 for idx, char in enumerate('abcdefghijklmnopqrstuvwxyz')}
    char_to_idx[' '] = 0
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}
    indices = torch.tensor([char_to_idx[char] for char in s], dtype=torch.long)
     # Move inputs to the correct device (the same device as the model)
    device = next(self.parameters()).device
    one_hot_vector = idx_to_onehot(indices).float().unsqueeze(0).to(device)  # (1, seq_length, N+1)    
    delay_tensor = torch.tensor([delay], dtype=torch.long, device=device)
    output = self.forward(one_hot_vector, delay_tensor)
    output_idx = torch.argmax(output, dim=-1).squeeze(0)

    return ''.join([idx_to_char[idx.item()] for idx in output_idx])



In [76]:
def test_variable_delay_model(model, max_delay=12):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  delay = random.randint(0, max_delay)

  total = 0
  correct = 0
  for i in range(1000):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s, delay)
    for c1, c2 in zip(s[:-delay], result[delay:]):
      correct += int(c1 == c2)
    total += len(s) - delay

  return correct / total

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [115]:
import time
start_time = time.time()

MAX_DELAY = 12
SEQ_LENGTH = 20


from tqdm.notebook import tqdm
from torch import nn

HIDDEN_SIZE=512 # TODO
BATCH_SIZE=64 #TODO
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 4

def train_variable_delay_model(model, dataloader):
  # TODO: implement training
  model.to(device)
  model.train()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.01)
  for epoch in range(num_epochs):
    epoch_loss = 0.0
        
    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False) as pbar:
      for i, (inputs, delays, targets) in enumerate(pbar):
        inputs = idx_to_onehot(inputs).to(device)  
        targets = targets.long().to(device)  # type for CrossEntropyLoss
        delays = delays.long().to(device)
                
        optimizer.zero_grad()
        outputs = model(inputs, delays)
                
        outputs = outputs.view(-1, N+1) 
        targets = targets.view(-1)  # flatten
                
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        # Update running loss here
        running_loss = loss.item()
        epoch_loss += running_loss
        pbar.set_postfix({'epoch_loss': epoch_loss / (i + 1)})
        
    

model = VariableDelayGRUMemory(hidden_size=HIDDEN_SIZE, max_delay=MAX_DELAY)
ds = VariableDelayEchoDataset(max_delay=MAX_DELAY, size=600000, seq_length=SEQ_LENGTH)
dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)
train_variable_delay_model(model, dataloader)

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
accuracy = test_variable_delay_model(model, max_delay=MAX_DELAY)
print(accuracy)
assert accuracy > 0.95, 'accuracy is too low'
print('tests passed')

Epoch 1/4:   0%|          | 0/9375 [00:00<?, ?it/s]

KeyboardInterrupt: 